In [1]:
import pandas as pd
import numpy as np


In [80]:
books = pd.read_csv('/Users/young/dataset_local/goodbook/books.csv',encoding="ISO-8859-1")
ratings = pd.read_csv('/Users/young/dataset_local/goodbook/ratings.csv',encoding="ISO-8859-1")
tags = pd.read_csv('/Users/young/dataset_local/goodbook/tags.csv')
book_tags = pd.read_csv('/Users/young/dataset_local/goodbook/book_tags.csv',encoding="ISO-8859-1")
to_read = pd.read_csv('/Users/young/dataset_local/goodbook/to_read.csv')


### 책의 태그를 활용해 유사한 책 찾기

In [81]:
tags_join_df = pd.merge(book_tags,tags,on='tag_id')

from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer(analyzer='word',ngram_range=(1,2),min_df=0,stop_words='english')
tfidf_matrix = tf.fit_transform(books['authors'])
tfidf_matrix

from sklearn.metrics.pairwise import linear_kernel

cosine_sim = linear_kernel(tfidf_matrix,tfidf_matrix)

titles = books['title']
indices = pd.Series(books.index,index=books['title'])
cosine_sim[indices['The Hobbit']]

sim_scores = list(enumerate(cosine_sim[indices['The Hobbit']]))
sim_scores = sorted(sim_scores,key=lambda x: x[1],reverse=True)
sim_scores[1:11]
book_indices = [i[0] for i in sim_scores]
titles.iloc[book_indices]

6                                              The Hobbit
18      The Fellowship of the Ring (The Lord of the Ri...
154            The Two Towers (The Lord of the Rings, #2)
160     The Return of the King (The Lord of the Rings,...
188     The Lord of the Rings (The Lord of the Rings, ...
                              ...                        
9995                            Bayou Moon (The Edge, #2)
9996    Means of Ascent (The Years of Lyndon Johnson, #2)
9997                                The Mauritius Command
9998    Cinderella Ate My Daughter: Dispatches from th...
9999                                  The First World War
Name: title, Length: 10000, dtype: object

In [85]:
books_with_tags = pd.merge(books, tags_join_df, left_on='book_id',right_on='goodreads_book_id',how='inner')
books_with_tags.head()

tf1 = TfidfVectorizer(analyzer='word',ngram_range=(1,2),min_df=0,stop_words='english')
tfidf_matrix_1 = tf1.fit_transform(books_with_tags['tag_name'].head(10000))
cosine_sim1 = linear_kernel(tfidf_matrix_1,tfidf_matrix_1)

titles1 = books['title']
indices1 = pd.Series(books.index, index=books['title'])

def tags_recommendations(title):
    idx = indices1[title]
    sim_scores = list(enumerate(cosine_sim1[idx]))
    sim_scores = sorted(sim_scores,key=lambda x : x[1],reverse=True)
    sim_scores = sim_scores[1:11]

    book_indices = [i[0] for i in sim_scores]

    print(sim_scores)
    print(book_indices)
    return titles.iloc[book_indices]

In [86]:
tags_recommendations('The Hobbit')

[(16, 1.0), (31, 1.0), (107, 1.0), (125, 1.0), (149, 1.0), (206, 1.0), (214, 1.0), (231, 1.0), (253, 1.0), (313, 1.0)]
[16, 31, 107, 125, 149, 206, 214, 231, 253, 313]


16             Catching Fire (The Hunger Games, #2)
31                                  Of Mice and Men
107    Confessions of a Shopaholic (Shopaholic, #1)
125                       Dune (Dune Chronicles #1)
149                                    The Red Tent
206          One for the Money (Stephanie Plum, #1)
214                                Ready Player One
231             The Gunslinger (The Dark Tower, #1)
253          Shiver (The Wolves of Mercy Falls, #1)
313                         Inkheart (Inkworld, #1)
Name: title, dtype: object

In [60]:
temp_df = books_with_tags.groupby('book_id')['tag_name'].apply(' '.join).reset_index()
temp_df.head()

,book_id,tag_name
0,1,to-read fantasy favorites currently-reading yo...
1,2,to-read fantasy favorites currently-reading yo...
2,3,to-read fantasy favorites currently-reading yo...
3,5,to-read fantasy favorites currently-reading yo...
4,6,to-read fantasy young-adult fiction harry-pott...


In [61]:
books = pd.merge(books,temp_df,left_on='book_id',right_on='book_id',how='inner')
books.head()

,id,book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url,tag_name
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...,to-read fantasy favorites currently-reading yo...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPrÃ©",1997.0,Harry Potter and the Philosopher's Stone,...,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...,to-read fantasy favorites currently-reading yo...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,...,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...,to-read fantasy favorites currently-reading yo...
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,...,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...,to-read favorites currently-reading young-adul...
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,...,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...,to-read favorites currently-reading young-adul...


In [62]:
books['corpus'] = (pd.Series(books[['authors','tag_name']].fillna('').values.tolist()).str.join(' '))
books['corpus'][:3]


0    Suzanne Collins to-read fantasy favorites curr...
1    J.K. Rowling, Mary GrandPrÃ© to-read fantasy f...
2    Stephenie Meyer to-read fantasy favorites curr...
Name: corpus, dtype: object

In [66]:
tf_corpus = TfidfVectorizer(analyzer='word',ngram_range=(1,2),stop_words='english',min_df=0)
tfidf_matrix_2 = tf_corpus.fit_transform(books['corpus'])
cosine_corpus = linear_kernel(tfidf_matrix_2,tfidf_matrix_2)

titles = books['title']
indices_2 = pd.Series(books.index, index=books['title'])

def corpus_recommendation(title):
    idx = indices_2[title]
    sim_scores = list(enumerate(cosine_corpus[idx]))
    sim_scores = sorted(sim_scores,key=lambda x : x[1],reverse=True)
    sim_scores = sim_scores[1:11]
    book_indices = [i[0] for i in sim_scores]
    return titles.iloc[book_indices]

corpus_recommendation('The Hobbit')

188     The Lord of the Rings (The Lord of the Rings, ...
154            The Two Towers (The Lord of the Rings, #2)
160     The Return of the King (The Lord of the Rings,...
18      The Fellowship of the Ring (The Lord of the Ri...
610              The Silmarillion (Middle-Earth Universe)
4975        Unfinished Tales of NÃºmenor and Middle-Earth
2308                               The Children of HÃºrin
963     J.R.R. Tolkien 4-Book Boxed Set: The Hobbit an...
465                             The Hobbit: Graphic Novel
8271                   The Complete Guide to Middle-Earth
Name: title, dtype: object